<a href="https://colab.research.google.com/github/11223344-khan/11223344-khan/blob/main/SER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Some Important Installations

In [ ]:
!pip install resampy==0.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320706 sha256=1119b7918fe6f0609247dee62a0edd61a914c690ebd708a617cde87cf076617c
  Stored in directory: /root/.cache/pip/wheels/e5/a0/79/29e61754e5b3941ad4c7d01bf5bea99768e64e4bdd3180f32b
Successfully built resampy


In [ ]:
!pip install librosa keras

In [ ]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.6 MB/s eta 0:00:00


# Import Necessary Libraries

In [ ]:
import os
import joblib
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
import resampy
from google.colab import drive
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import ipywidgets as widgets
import os
import librosa
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Mount the Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Ravdess dataset and preprocess audio files

In [ ]:
# Load Ravdess dataset
Ravdess = pd.DataFrame({
    'emotions': [],
    'path': []
})

scaler = MinMaxScaler(feature_range=(0, 1))

Ravdess_dir = "/content/drive/MyDrive/Audio_Recognition/Ravedess"
for actor_folder in os.listdir(Ravdess_dir):
    actor_folder_path = os.path.join(Ravdess_dir, actor_folder)
    for file_name in os.listdir(actor_folder_path):
        file_path = os.path.join(actor_folder_path, file_name)
        emo_id = file_name.split('-')[2]
        if emo_id == '01':
            emo = 'neutral'
        elif emo_id == '02':
            emo = 'calm'
        elif emo_id == '03':
            emo = 'happy'
        elif emo_id == '04':
            emo = 'sad'
        elif emo_id == '05':
            emo = 'angry'
        elif emo_id == '06':
            emo = 'fearful'
        elif emo_id == '07':
            emo = 'disgust'
        else:
            emo = 'surprised'
        new_data = pd.DataFrame({'emotions': [emo], 'path': [file_path]})
        Ravdess = pd.concat([Ravdess, new_data], ignore_index=True)

In [ ]:
Ravdess

,emotions,path
0,calm,/content/drive/MyDrive/Audio_Recognition/Raved...
1,neutral,/content/drive/MyDrive/Audio_Recognition/Raved...
2,calm,/content/drive/MyDrive/Audio_Recognition/Raved...
3,calm,/content/drive/MyDrive/Audio_Recognition/Raved...
4,calm,/content/drive/MyDrive/Audio_Recognition/Raved...
...,...,...
1435,sad,/content/drive/MyDrive/Audio_Recognition/Raved...
1436,surprised,/content/drive/MyDrive/Audio_Recognition/Raved...
1437,angry,/content/drive/MyDrive/Audio_Recognition/Raved...
1438,angry,/content/drive/MyDrive/Audio_Recognition/Raved...


# Display Emotion Distribution in Ravdess Dataset

In [ ]:
Ravdess['emotions'].value_counts()

calm         192
happy        192
disgust      192
fearful      192
sad          192
surprised    192
angry        192
neutral       96
Name: emotions, dtype: int64

# Define and List Audio Files in the Directory

In [ ]:
# Define the path to your audio files directory
audio_files_dir = '/content/drive/MyDrive/Audio_Recognition/Ravedess'

# List audio files in the directory
audio_files = [os.path.join(audio_files_dir, file) for file in os.listdir(audio_files_dir)]

In [ ]:
# List the contents of the dataset folder.
os.listdir(audio_files_dir)

['Actor_20',
 'Actor_17',
 'Actor_23',
 'Actor_15',
 'Actor_22',
 'Actor_18',
 'Actor_21',
 'Actor_19',
 'Actor_16',
 'Actor_24',
 'Actor_09',
 'Actor_10',
 'Actor_05',
 'Actor_08',
 'Actor_12',
 'Actor_06',
 'Actor_11',
 'Actor_07',
 'Actor_14',
 'Actor_13',
 'Actor_01',
 'Actor_04',
 'Actor_02',
 'Actor_03']

# Function to Extract Audio Features
Function extract_features that takes a file path as input and extracts audio features, including (MFCC), chroma feature, and mel spectrogram features. The function uses the librosa library to load the audio file, compute the necessary features, and returns a concatenated array of these features.

In [ ]:
# Function to extract features from audio files
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    with np.errstate(divide='ignore', invalid='ignore'):
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        if chroma:
            stft = np.abs(librosa.stft(audio))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=2048, hop_length=512).T, axis=0)
            result = np.hstack((result, mel))
    return result

# Function to Preprocess Audio Data and Extract Features
This code defines a function preprocess_data that takes the directory path containing audio files, iterates through each file, extracts emotion labels and audio features using the extract_features function, and returns two lists.

In [ ]:
# Function to preprocess and extract features from a list of audio files
def preprocess_data(audio_files_dir):
    emotions = []
    features = []
    for actor_folder in os.listdir(audio_files_dir):
        actor_folder_path = os.path.join(audio_files_dir, actor_folder)
        for file_name in os.listdir(actor_folder_path):
            file_path = os.path.join(actor_folder_path, file_name)
            emotion = file_name.split('-')[2]
            if emotion not in emotions:
                emotions.append(emotion)
            feature = extract_features(file_path)
            features.append([feature, emotion])
    return emotions, features

# Load and Preprocess Audio Data
This code calls the preprocess_data function to load audio data from the specified directory (audio_files_dir). It extracts emotion labels and audio features for each file, storing the unique emotion labels in the emotions list and the corresponding feature-emotion pairs in the features list.

In [ ]:
# Load and preprocess data
emotions, features = preprocess_data(audio_files_dir)

# Splitting the Dataset

In [ ]:
# Split the dataset into training and testing sets
X = np.array([x[0] for x in features])
y = np.array([y[1] for y in features])

In [ ]:
X

array([[-8.52922668e+02,  3.84620819e+01, -4.90387869e+00, ...,
         8.31060873e-08,  1.16610686e-08,  1.39717815e-09],
       [-7.13109070e+02,  3.78296471e+01, -1.04684496e+01, ...,
         7.25710265e-07,  1.32387200e-07,  1.47552113e-08],
       [-7.95232117e+02,  4.12144318e+01, -1.22606478e+01, ...,
         5.21315862e-08,  1.36715341e-08,  1.11911969e-09],
       ...,
       [-3.25668671e+02,  4.84727020e+01, -1.98268089e+01, ...,
         1.35883602e-04,  2.19930935e-05,  2.14702618e-06],
       [-3.31223969e+02,  4.62784882e+01, -1.89334869e+01, ...,
         1.32261193e-04,  1.91710824e-05,  1.73276806e-06],
       [-5.81411987e+02,  6.99668579e+01, -7.83518362e+00, ...,
         6.76299851e-06,  1.65709207e-06,  1.77608470e-07]])

In [ ]:
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

# Label Encoding and One-Hot Encoding for Emotion Labels
LabelEncoder (le) is used to transform the categorical emotion labels (y) into numerical representations. The fit_transform method of the LabelEncoder is applied to the emotion labels, and then to_categorical is used to perform one-hot encoding on the transformed labels, converting them into binary vectors.







In [ ]:
# Label encoding and one-hot encoding
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshaping the Data

In [ ]:
# Reshape the data for CNN input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Normalize Input Data using StandardScaler
This code utilizes the StandardScaler (scaler) to normalize the input data. The fit_transform method is applied to the training data (X_train) to calculate the mean and standard deviation and perform the normalization. The same transformation is then applied to the test data (X_test).

In [ ]:
# Normalize the input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [ ]:
# Normalize the input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Define and Train Convolutional Neural Network (CNN) Model
This code defines a Convolutional Neural Network (CNN) model using the Keras Sequential API. The model consists of convolutional layers with max pooling, dropout layers for regularization, and dense layers. The model is compiled with the categorical crossentropy loss function and the Adam optimizer. During training, the ModelCheckpoint callback is used to save the best model based on validation accuracy. The model is trained on the training data (X_train, y_train) for 100 epochs with a batch size of 32.

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(emotions), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Save the best model during training
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1, callbacks=[checkpoint])

Epoch 1/100
36/36 [==============================] - 4s 74ms/step - loss: 2.0276 - accuracy: 0.1832 - val_loss: 1.9668 - val_accuracy: 0.2188
Epoch 2/100
 1/36 [..............................] - ETA: 2s - loss: 2.0014 - accuracy: 0.1875

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 3s 75ms/step - loss: 1.9087 - accuracy: 0.2439 - val_loss: 1.8498 - val_accuracy: 0.3368
Epoch 3/100
36/36 [==============================] - 2s 64ms/step - loss: 1.8431 - accuracy: 0.2830 - val_loss: 1.8127 - val_accuracy: 0.3368
Epoch 4/100
36/36 [==============================] - 2s 49ms/step - loss: 1.7810 - accuracy: 0.3021 - val_loss: 1.7539 - val_accuracy: 0.3715
Epoch 5/100
36/36 [==============================] - 2s 46ms/step - loss: 1.7622 - accuracy: 0.3064 - val_loss: 1.7092 - val_accuracy: 0.3646
Epoch 6/100
36/36 [==============================] - 2s 44ms/step - loss: 1.6898 - accuracy: 0.3585 - val_loss: 1.7330 - val_accuracy: 0.3056
Epoch 7/100
36/36 [==============================] - 2s 42ms/step - loss: 1.6628 - accuracy: 0.3698 - val_loss: 1.7073 - val_accuracy: 0.3472
Epoch 8/100
36/36 [==============================] - 2s 42ms/step - loss: 1.6450 - accuracy: 0.3663 - val_loss: 1.7032 - val_accuracy: 0.3507
Epoch 9/100
36/36 

# Load the best model

In [ ]:
# Load the best model for evaluation
model.load_weights("best_model.h5")

# Evaluation

In [ ]:
# Evaluate the model on the test set
_, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

9/9 [==============================] - 0s 11ms/step - loss: 1.1208 - accuracy: 0.7153
Test Accuracy: 71.53%


# Save the Model

In [ ]:
# Save the model to a file
joblib.dump(model, 'saved_model.joblib')

['saved_model.joblib']

# Load the Model

In [ ]:
# Load the model from the file
loaded_model = joblib.load('saved_model.joblib')

In [ ]:
# Load the trained model
model = load_model("best_model.h5")

# Load the scaler used for normalization during training
scaler = StandardScaler()

# Extract Features from User Input Audio
Defines a function extract_user_input_features to extract audio features, including MFCC, chroma, and mel spectrogram features, from a user's input audio file using the librosa library. The function is designed for efficient feature extraction from user-provided audio for analysis or model predictions.

In [ ]:
# Function to extract features from user input audio file
def extract_user_input_features(file_path, mfcc=True, chroma=True, mel=True):
    with np.errstate(divide='ignore', invalid='ignore'):
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        if chroma:
            stft = np.abs(librosa.stft(audio))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=2048, hop_length=512).T, axis=0)
            result = np.hstack((result, mel))
    return result

# Predict Emotion from User Input Audio
This function predicts the emotion label from a user's input audio file. It extracts features, reshapes, normalizes using a specified scaler, and utilizes a pre-trained CNN model to make predictions. The result is the predicted emotion label based on the highest model probability.

In [ ]:
# Function to predict emotion from user input
def predict_emotion(file_path, scaler):
    # Extract features from user input
    user_input_features = extract_user_input_features(file_path)

    # Reshape and normalize the input data
    user_input_features = user_input_features.reshape(1, user_input_features.shape[0], 1)
    user_input_features = scaler.transform(user_input_features.reshape(-1, user_input_features.shape[-1])).reshape(user_input_features.shape)

    # Make a prediction
    prediction = model.predict(user_input_features)

    # Get the emotion label
    emotion_label = np.argmax(prediction)

    return emotion_label

# Convert Emotion Label to Text
The function emotion_to_text that takes an emotion label and a list of emotions, and returns the corresponding text representation of the emotion. If the provided emotion label is out of bounds, it returns "Unknown Emotion." This function is useful for converting numerical emotion labels to human-readable text.

In [ ]:
# Function to convert emotion label to text
def emotion_to_text(emotion_label, emotions):
    try:
        emotion_text = emotions[emotion_label]
    except IndexError:
        emotion_text = "Unknown Emotion"
    return emotion_text

# Fit the Scalar

In [ ]:
# Fit and save the scaler
scaler.fit(X_train.reshape(-1, X_train.shape[-1]))

# Save the scaler to a file
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename)

# Save the trained model
model.save("best_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Continuous Emotion Prediction from User Input
This code snippet creates an infinite loop prompting the user to enter the path to an audio file. It predicts the emotion from the user's input using the pre-trained model, converts the numerical prediction to text, and displays the result. The loop continues until the user enters 'exit' to quit. Any errors during the process are also handled and displayed.

In [ ]:
# Infinite loop to keep asking the user for input
while True:
    # Get the user's voice input file path
    user_input_file_path = input("Enter the path to the audio file ('exit' to quit): ")

    # Check if the user wants to exit
    if user_input_file_path.lower() == 'exit':
        break

    try:
        # Predict emotion and convert to text
        # Assuming you have the list of emotions as specified in your dataset
        dataset_emotions = ["angry", "calm", "disgust", "fearful", "happy", "sad", "surprised", "neutral"]

        # Predict emotion and convert to text
        predicted_emotion_label = predict_emotion(user_input_file_path, scaler)
        predicted_emotion_text = emotion_to_text(predicted_emotion_label, dataset_emotions)

        # Display the result
        print(f"Predicted Emotion: {predicted_emotion_text}")


    except Exception as e:
        print(f"Error: {e}")

Enter the path to the audio file ('exit' to quit): /content/drive/MyDrive/Audio_Recognition/Ravedess/Actor_10/03-01-04-01-01-01-10.wav
1/1 [==============================] - 0s 38ms/step
Predicted Emotion: fearful
Enter the path to the audio file ('exit' to quit): /content/drive/MyDrive/Audio_Recognition/Ravedess/Actor_17/03-01-01-01-01-02-17.wav
1/1 [==============================] - 0s 36ms/step
Predicted Emotion: calm
Enter the path to the audio file ('exit' to quit): /content/drive/MyDrive/Audio_Recognition/Ravedess/Actor_21/03-01-01-01-01-01-21.wav
1/1 [==============================] - 0s 32ms/step
Predicted Emotion: calm
Enter the path to the audio file ('exit' to quit): quit


<ipython-input-43-8e798c0804e0>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error: [Errno 2] No such file or directory: 'quit'
Enter the path to the audio file ('exit' to quit): exit


# Upload and Classify Audio File with Pre-trained Model
This code snippet loads a pre-trained neural network model (best_model.h5) and the associated scaler used during training. It defines a function triggered by a button click to upload an audio file, predict its emotion using the loaded model, and display the result. The button is created using the ipywidgets library, allowing users to interactively upload and classify audio files.

In [ ]:
# Load the trained model
model = load_model("best_model.h5")

# Load the scaler used for normalization during training
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename)

# Define the function that will be called when the button is clicked
def on_button_click(b):
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    tmp_path = f'/tmp/{file_name}'
    os.rename(file_name, tmp_path)
    global uploaded_file_path
    uploaded_file_path = tmp_path
    print(f"File uploaded to {uploaded_file_path}")

    # Predict emotion and convert to text
    dataset_emotions = ["angry", "calm", "disgust", "fearful", "happy", "sad", "surprised", "neutral"]
    predicted_emotion_label = predict_emotion(uploaded_file_path, scaler)
    predicted_emotion_text = emotion_to_text(predicted_emotion_label, dataset_emotions)

    # Display the result
    print(f"Predicted Emotion: {predicted_emotion_text}")

# Function to predict emotion from user input
def predict_emotion(file_path, scaler):
    # Extract features from user input
    user_input_features = extract_user_input_features(file_path)

    # Reshape and normalize the input data
    user_input_features = user_input_features.reshape(1, user_input_features.shape[0], 1)
    user_input_features = scaler.transform(user_input_features.reshape(-1, user_input_features.shape[-1])).reshape(user_input_features.shape)

    # Make a prediction
    prediction = model.predict(user_input_features)

    # Get the emotion label
    emotion_label = np.argmax(prediction)

    return emotion_label

# Function to convert emotion label to text
def emotion_to_text(emotion_label, emotions):
    try:
        emotion_text = emotions[emotion_label]
    except IndexError:
        emotion_text = "Unknown Emotion"
    return emotion_text

# Create a button widget
button = widgets.Button(description="Upload and Classify File")

# Attach the function to the button's click event
button.on_click(on_button_click)

# Display the button
display(button)

Button(description='Upload and Classify File', style=ButtonStyle())

Saving 03-01-01-01-02-02-03.wav to 03-01-01-01-02-02-03.wav
File uploaded to /tmp/03-01-01-01-02-02-03.wav
1/1 [==============================] - 0s 116ms/step
Predicted Emotion: calm


# Save the Model to Drive

In [ ]:
# Save the trained model to Google Drive
model.save("/content/drive/MyDrive/Audio_Recognition/best_model.h5")

# Save the scaler to Google Drive
scaler_filename = "/content/drive/MyDrive/Audio_Recognition/scaler.save"
joblib.dump(scaler, scaler_filename)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['/content/drive/MyDrive/Audio_Recognition/scaler.save']